# Deep Convolutional Q-Learning Pacman

In [2]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import deque
from torch.utils.data import DataLoader, TensorDataset

## Build an AI
![Description](https://gymnasium.farama.org/_images/pacman.gif)

In [3]:
class Network(nn.Module):
    
    def __init__(self, action_size, seed = 42):
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.conv1 = nn.Conv2d(3, 32, kernel_size=8, stride=4)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(64)
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.bn4 = nn.BatchNorm2d(128)
        self.fc1 = nn.Linear(128 * 10 * 10, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, action_size)
        
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.relu(self.bn4(self.conv4(x)))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    

## Training the AI

In [4]:
import gymnasium as gym
env = gym.make('MsPacmanDeterministic-v4', full_action_space=False)
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
print('State shape:', state_shape)
print('State size:', state_size)
print('Action size:', action_size)
print("Name of the actions: ", env.unwrapped.get_action_meanings())

State shape: (210, 160, 3)
State size: 210
Action size: 9
Name of the actions:  ['NOOP', 'UP', 'RIGHT', 'LEFT', 'DOWN', 'UPRIGHT', 'UPLEFT', 'DOWNRIGHT', 'DOWNLEFT']


A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
[Powered by Stella]


In [5]:
# Initializing the hyperparameters
learning_rate = 5e-4
minibatch_size = 64
discount_factor = 0.99

In [6]:
# Processing the frames to make them suitable for the network
from PIL import Image
from torchvision import transforms

def process_frame(frame):
    frame = Image.fromarray(frame)
    preprocess = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor()
    ])
    return preprocess(frame).unsqueeze(0)

In [7]:
# Implementing the DCQN algorithm
class Agent:
    
    def __init__(self, action_size, seed = 42):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.action_size = action_size
        self.memory = deque(maxlen=10000)
        self.local_qnetwork = Network( action_size).to(self.device)
        self.target_qnetwork = Network( action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr=learning_rate)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        state = process_frame(state)
        next_state = process_frame(next_state)
        self.memory.append((state, action, reward, next_state, done))
        if len(self.memory) > minibatch_size:
            experiences = random.sample(self.memory, k=minibatch_size)
            self.learn(experiences, discount_factor)
    
    def act(self, state, epsilon = 0.01):
        state = process_frame(state).to(self.device)
        self.local_qnetwork.eval()
        with torch.no_grad():
            action_values = self.local_qnetwork(state)
        self.local_qnetwork.train()
        if random.random() > epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
    
    def learn(self, experiences, discount_factor):
        states, actions, rewards, next_states, dones = zip(*experiences)
        states = torch.from_numpy(np.vstack(states)).float().to(self.device)
        actions = torch.from_numpy(np.vstack(actions)).long().to(self.device)
        rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
        next_states = torch.from_numpy(np.vstack(next_states)).float().to(self.device)
        dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(self.device)
        next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
        q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
        q_expected = self.local_qnetwork(states).gather(1, actions)
        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

In [8]:
# Initializing the DCQN agent
agent = Agent(action_size)

In [17]:
# Training the agent
number_episodes = 2000
maximum_number_time_steps_per_episode = 10000
epsilon_starting_value = 1.0
epsilon_end_value = 0.01
epsilon_decay_rate = 0.995
epsilon = epsilon_starting_value
scores_on_100_episodes = deque(maxlen=100)

for episode in range(1, number_episodes+1):
    state, _ = env.reset()
    score = 0
    for time_step in range(maximum_number_time_steps_per_episode):
        action = agent.act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break

    scores_on_100_episodes.append(score)
    epsilon = max(epsilon_end_value, epsilon_decay_rate*epsilon)

    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end="")
    if episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))

    if np.mean(scores_on_100_episodes) >= 500.0:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode-100, np.mean(scores_on_100_episodes)))
        torch.save(agent.local_qnetwork.state_dict(), 'pacman_deep_convolutional_q_learning.pth')
        break

Episode 100	Average Score: 274.50
Episode 200	Average Score: 375.50
Episode 300	Average Score: 416.50
Episode 400	Average Score: 460.30
Episode 482	Average Score: 501.30
Environment solved in 382 episodes!	Average Score: 501.30


In [10]:
import imageio
from datetime import datetime
# from gym.wrappers.monitoring.video_recorder import VideoRecorder

# Load the model with the best performance
agent.local_qnetwork.load_state_dict(torch.load('pacman_deep_convolutional_q_learning.pth', weights_only=False))

# Initialize the environment
def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frame = frame[:frame.shape[0] - frame.shape[0] % 16, :frame.shape[1] - frame.shape[1] % 16]
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action)
    env.close()
    
    now = datetime.now().strftime('%b%d_%H-%M-%S')
    video_filename = f'video_{env_name}_{now}.mp4'
    
    imageio.mimsave(video_filename, frames, fps=30, quality=10)

show_video_of_model(agent, 'MsPacman-v4')